In [2]:
## import packages and helper functions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import keras
import tensorflow as tf

from helper import *
from connection import *

In [87]:
## build autoencoder
from keras import layers

n_domain = 14 # number of domains

encoding_dim = 10
## input n_domain * 2 to represent missing values
input_layer = keras.Input(shape=(n_domain * 2,))
encode_layer = layers.Dense(encoding_dim, activation="relu")(input_layer)
decode_layer = layers.Dense(n_domain*2, activation="sigmoid")(encode_layer)

autoencoder = keras.Model(input_layer, decode_layer)

In [88]:
# build encoder and decoder model
encoder = keras.Model(input_layer, encode_layer)
encoded_input = keras.Input(shape=(encoding_dim,))
decode_layer = autoencoder.layers[-1]
decoder = keras.Model(encoded_input, decode_layer(encoded_input))

In [89]:
# compile with optimizer and loss function
autoencoder.compile(optimizer="adam", loss="mse")

In [29]:
# import data
df = pd.read_csv("data/raw_percentile.csv")
df

,id,patient_id,task_type_id,task_level,completed_task_count,accuracy,condition_since,birth_year,start_time,start_time_min,end_time_min,deficit_id,disorder_id,domain_id,progression_order,percentile
0,639141,15796,20,1,2,0.032,5y,1950,2014-08-02,2014-08-02 21:11:37,2014-08-02 21:11:40,"{1, 2, 3, 4, 8}","{2, 3}",1,0,0.010932
1,659924,16441,20,1,3,1.000,6m,1959,2014-08-08,2014-08-08 16:21:14,2014-08-08 16:21:54,"{1, 2, 3, 4}",{2},1,0,0.010932
2,677465,16441,20,1,3,1.000,6m,1959,2014-08-13,2014-08-13 01:07:54,2014-08-13 01:08:17,"{1, 2, 3, 4}",{2},1,0,0.010932
3,677586,16441,20,1,3,1.000,6m,1959,2014-08-13,2014-08-13 01:36:48,2014-08-13 01:37:13,"{1, 2, 3, 4}",{2},1,0,0.010932
4,679091,16441,20,1,3,1.000,6m,1959,2014-08-13,2014-08-13 16:15:30,2014-08-14 02:40:50,"{1, 2, 3, 4}",{2},1,0,0.010932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1601853,4662260,82474,75,1,10,0.780,1y,1983,2016-09-20,2016-09-20 18:52:46,2016-09-20 18:57:20,"{1, 2, 3, 4}","{2, 3}",8,3,0.328086
1601854,4682259,82474,75,1,10,0.870,1y,1983,2016-09-22,2016-09-22 17:29:06,2016-09-22 17:33:08,"{1, 2, 3, 4}","{2, 3}",8,3,0.328086
1601855,4698670,82474,75,1,10,0.900,1y,1983,2016-09-24,2016-09-24 15:58:14,2016-09-24 16:03:42,"{1, 2, 3, 4}","{2, 3}",8,3,0.328086
1601856,4704074,82474,75,1,10,0.900,1y,1983,2016-09-25,2016-09-25 14:45:49,2016-09-25 14:53:32,"{1, 2, 3, 4}","{2, 3}",8,3,0.328086


# build training data

In [15]:
# make column name array
def column_name_f(n):
    output = ["info"]
    for i in range(1, n+1):
        output.append("domain %d true" %(i))
        output.append("domain %d comp" %(i))
    return output

In [60]:
# given a patient's data, create a dataframe of scores, patient id
def create_training_data(data):
    data = data.sort_values(by="end_time_min").reset_index() # sort data
    n_sessions = data.shape[0]
    n_domains = 14
    output_array = np.zeros((1 + n_domains * 2, n_sessions))
    i = 0

    for idx, row in data.iterrows():
        d = row["domain_id"]
        p = row["percentile"]

        # update output array
        if i != 0: output_array[:, i] = output_array[:, i-1]
        output_array[0, i] = row["patient_id"]
        output_array[d*2-1, i] = p # score
        output_array[d*2, i] = 1-p # 1-score

        i += 1
    return output_array

In [65]:
# takes about ~2 min to run
train_X = np.array([])
df_groups = df.groupby("patient_id")
df_groups.groups
keys = df_groups.groups.keys()
for key in keys:
    data = df_groups.get_group(key)
    temp = create_training_data(data)
    if train_X.size == 0:
        train_X = temp
    else:
        train_X = np.hstack((train_X, temp)) 
train_X

array([[1.41890000e+04, 1.41890000e+04, 1.41890000e+04, ...,
        1.08830000e+05, 1.99388000e+05, 1.99388000e+05],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        7.59951825e-01, 0.00000000e+00, 7.59951825e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        2.40048175e-01, 0.00000000e+00, 2.40048175e-01],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.34256173e-03, 4.33580672e-01, 4.33580672e-01, ...,
        7.78322629e-01, 0.00000000e+00, 0.00000000e+00],
       [9.92657438e-01, 5.66419328e-01, 5.66419328e-01, ...,
        2.21677371e-01, 0.00000000e+00, 0.00000000e+00]])

In [67]:
temp_df = pd.DataFrame(train_X)
temp_df.to_csv("data/autoencoder_train_1.csv", index=False)

In [73]:
np.transpose(train_X).shape

(1601858, 29)

# Training

In [92]:
train = np.transpose(train_X)
train = train_X[:, 1:]
train.shape

(1601858, 28)

In [96]:
from sklearn.model_selection import train_test_split
x_train, x_test = train_test_split(train, test_size=0.20, random_state=42)
x_train, x_val = train_test_split(x_train, test_size=0.20, random_state=42)

In [97]:
autoencoder.fit(x_train, x_train,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(x_val, x_val))

Epoch 1/100
4005/4005 [==============================] - 2s 448us/step - loss: 0.0221 - val_loss: 0.0220
Epoch 2/100
4005/4005 [==============================] - 2s 421us/step - loss: 0.0221 - val_loss: 0.0220
Epoch 3/100
4005/4005 [==============================] - 2s 417us/step - loss: 0.0221 - val_loss: 0.0220
Epoch 4/100
4005/4005 [==============================] - 2s 403us/step - loss: 0.0221 - val_loss: 0.0220
Epoch 5/100
4005/4005 [==============================] - 2s 410us/step - loss: 0.0221 - val_loss: 0.0220
Epoch 6/100
4005/4005 [==============================] - 2s 398us/step - loss: 0.0221 - val_loss: 0.0221
Epoch 7/100
4005/4005 [==============================] - 2s 400us/step - loss: 0.0221 - val_loss: 0.0220
Epoch 8/100
4005/4005 [==============================] - 2s 398us/step - loss: 0.0221 - val_loss: 0.0220
Epoch 9/100
4005/4005 [==============================] - 2s 401us/step - loss: 0.0221 - val_loss: 0.0220
Epoch 10/100
4005/4005 [==============================]

# Testing

In [98]:
x_test_encoded = encoder.predict(x_test, batch_size=256)

1252/1252 [==============================] - 0s 209us/step


In [102]:
x_test.shape

(320372, 28)

In [101]:
x_test_encoded.shape

(320372, 10)

In [103]:
x_decoded = decoder.predict(x_test_encoded)

10012/10012 [==============================] - 2s 191us/step


In [104]:
x_decoded.shape

(320372, 28)

In [107]:
x_test[0, :]

array([0.75995182, 0.24004818, 0.        , 0.        , 0.47352006,
       0.52647994, 0.        , 0.        , 0.21298966, 0.78701034,
       0.29990384, 0.70009616, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.31128134, 0.68871866,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.63719172, 0.36280828])

In [108]:
x_decoded[0, :]

array([6.9029552e-01, 6.0489025e-02, 0.0000000e+00, 1.0108828e-34,
       6.1131096e-01, 3.5200322e-01, 1.4610444e-02, 1.6641770e-02,
       3.9018013e-02, 4.7832555e-01, 3.0466032e-01, 6.7708629e-01,
       1.6645545e-02, 3.0436989e-02, 1.3438554e-17, 0.0000000e+00,
       1.0178118e-10, 8.3955251e-02, 3.4518754e-01, 6.9202334e-01,
       5.1309091e-18, 6.2653277e-20, 1.4793537e-02, 4.9202293e-02,
       6.5008487e-23, 4.1087009e-02, 6.8994975e-01, 1.6583031e-01],
      dtype=float32)